In [1]:
import os
import json
import pandas as pd
from urllib.request  import urlretrieve
import gzip
import geopandas as gpd
import folium 
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [2]:
def run():
    ##透過這個網址可以下載最新的資料
    data_url='https://pm25.lass-net.org/data/last-all-airbox.json.gz'
    ##開始下載資料 後面是放存的路徑
    urlretrieve(data_url,r'D:\pm2.5\last-all-airbox.json.gz')

def read(path):
    with gzip.open(path,'rb') as f:
        last_data=f.read()
        data = json.loads(last_data)
    data_list=data['feeds']
    ##存資料的DataFrame
    data_taipei=pd.DataFrame(columns=['device_id','PM2.5','lat','lon'])
    ## i存沒有區域資訊的站有幾個

    for i in range(len(data_list)):
    ##抓出在台北的測站
        try:
            if data_list[i]['area']=='taipei':
                data_taipei.loc[i,'device_id']=data_list[i]['device_id']
                data_taipei.loc[i,'PM2.5']=data_list[i]['s_d0']
                data_taipei.loc[i,'lat']=data_list[i]['gps_lat']
                data_taipei.loc[i,'lon']=data_list[i]['gps_lon']

        except KeyError:
            pass
    data_taipei.reset_index(inplace=True)
    return data_taipei


def spatial_join(data):
    taipei=gpd.read_file(r'D:\pm2.5\taipei.geojson',encoding='utf-8')
    taipei=taipei.to_crs(epsg=4326)
    sites = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.lon, data.lat), crs="EPSG:4326")
    sites = gpd.sjoin(sites, taipei, how="inner", op='intersects')
    sites['PM2.5'] = sites['PM2.5'].apply(pd.to_numeric)
    sites=sites.groupby('id').mean()
    sites.reset_index(inplace=True)
    return sites


def central_x(point):
    xy=str(point).split(' ')
    return float(xy[1][1:])

def central_y(point):
    xy=str(point).split(' ')
    return float(xy[2].replace(")", ""))

#r'D:\pm2.5\taipei-centroids-4326.geojson'	
def centrality(path):
    df1=pd.DataFrame(columns=['lat','lon','PM2.5'])
    taipei=gpd.read_file(r'D:\pm2.5\taipei.geojson',encoding='utf-8')
    taipei_c=gpd.read_file(path,encoding='utf-8')
    taipei_c=taipei_c.to_crs(epsg=4326)
    df1['lat']=taipei_c['geometry'].apply(central_y)
    df1['lon']=taipei_c['geometry'].apply(central_x)
    df1['id']=taipei['id']
    return df1


def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r 

def idw(df):
    if len(df)==0:
        return np.nan
    else:
        numerator=sum(df['PM2.5']/(np.power(df['distance'],2)))
        denominator=sum(1/np.power(df['distance'],2))
        return round(numerator/denominator,2)

def pm_insert(central,data_taipei):
    t=[]
    for i in range(len(central)):
        data_taipei['distance']=data_taipei.apply(lambda x: haversine(central['lon'][i], central['lat'][i],x['lon'],x['lat']),axis=1)
        t.append(idw(data_taipei[data_taipei['distance']<3]))
    central['PM2.5']=t
    return central

def draw_map(data_taipei,df):
    taipei=gpd.read_file(r'D:\pm2.5\taipei.geojson',encoding='utf-8')
    m = folium.Map( location=[25.11282, 121.538],zoom_start=11,tiles='stamenterrain')
    tooltip = 'Click me!'
    for i in range(len(data_taipei)):
        df_site = pd.DataFrame(data=[[data_taipei['device_id'][i],data_taipei['PM2.5'][i]]], columns=['ID','PM2.5'])
        html = df_site.to_html(classes='table table-striped table-hover table-condensed table-responsive')
        popup = folium.Popup(html)
        folium.Marker([data_taipei.lat[i],data_taipei.lon[i]], popup=popup, tooltip=tooltip,icon=folium.Icon(color='red')).add_to(m)
    
    data_tp=pd.DataFrame()
    data_tp['ID']=taipei['id']
    data_tp['Value']= df['PM2.5']
    folium.Choropleth(
        geo_data=taipei,
        name='choropleth',
        data=data_tp,
        columns=['ID','Value'],
         fill_color='YlOrRd',key_on='feature.properties.id').add_to(m)

    folium.LayerControl().add_to(m)

    return m

In [3]:
import time

if __name__ == '__main__':
    tStart = time.time()
    run()
    data_taipei=read(r'D:\pm2.5\last-all-airbox.json.gz')
    sites=spatial_join(data_taipei)
    central_points=centrality(r'D:\pm2.5\taipei-centroids-4326.geojson')
    df=pm_insert(central_points,data_taipei)
    tEnd = time.time()#計時結束
    #列印結果
    print ("It cost %f sec" % (tEnd - tStart))
    draw_map(data_taipei,df).save(r'D:\pm2.5\PM2.5.html')
    

It cost 51.264917 sec


In [4]:

draw_map(data_taipei,df)